In [44]:
import pandas as pd
import hvplot.pandas
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import requests
import json


In [28]:
# Using the read_csv function and Path module, create a DataFrame 
# by importing the file from the Resources folder
Copper_df = pd.read_csv(
    Path('Resources/Copper.csv'),
    index_col='DATE',
    infer_datetime_format=True, 
    parse_dates=True
)
Nickel_df = pd.read_csv(
    Path('Resources/Nickel.csv'),
    index_col='DATE',
    infer_datetime_format=True, 
    parse_dates=True
)
Zinc_df = pd.read_csv(
    Path('Resources/Zinc.csv'),
    index_col='DATE',
    infer_datetime_format=True, 
    parse_dates=True
)


# Review the first and last five rows of the DataFrame

Copper_df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 383 entries, 1990-01-01 to 2021-11-01
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PCOPPUSDM  383 non-null    float64
dtypes: float64(1)
memory usage: 6.0 KB


In [29]:
Nickel_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 383 entries, 1990-01-01 to 2021-11-01
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PNICKUSDM  383 non-null    float64
dtypes: float64(1)
memory usage: 6.0 KB


In [30]:
Zinc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 383 entries, 1990-01-01 to 2021-11-01
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PZINCUSDM  383 non-null    float64
dtypes: float64(1)
memory usage: 6.0 KB


In [20]:
Copper_df.isna().any().sum()
Nickel_df.isna().any().sum()
Zinc_df.isna().any().sum()

0

In [37]:
Commodities_list = [Copper_df, Nickel_df, Zinc_df]
Commodities_df =pd.concat([Copper_df, Nickel_df, Zinc_df],axis=1)
display(Commodities_df)
Commodities_df.info()

,PCOPPUSDM,PNICKUSDM,PZINCUSDM
DATE,,,
1990-01-01,2365.556991,7056.00000,1294.000000
1990-02-01,2358.943400,6977.00000,1394.000000
1990-03-01,2625.702723,9267.00000,1666.000000
1990-04-01,2685.226891,8939.00000,1686.000000
1990-05-01,2740.342391,8698.00000,1775.000000
...,...,...,...
2021-07-01,9450.820455,18818.51136,2947.517273
2021-08-01,9370.138571,19141.29667,2987.947143
2021-09-01,9324.709545,19376.87500,3036.018636


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 383 entries, 1990-01-01 to 2021-11-01
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PCOPPUSDM  383 non-null    float64
 1   PNICKUSDM  383 non-null    float64
 2   PZINCUSDM  383 non-null    float64
dtypes: float64(3)
memory usage: 12.0 KB


In [38]:
Commodities_df.rename(columns={'PCOPPUSDM': 'Copper', 'PNICKUSDM': 'Nickel', 'PZINCUSDM': 'Zinc'}, inplace=True)
display(Commodities_df)

,Copper,Nickel,Zinc
DATE,,,
1990-01-01,2365.556991,7056.00000,1294.000000
1990-02-01,2358.943400,6977.00000,1394.000000
1990-03-01,2625.702723,9267.00000,1666.000000
1990-04-01,2685.226891,8939.00000,1686.000000
1990-05-01,2740.342391,8698.00000,1775.000000
...,...,...,...
2021-07-01,9450.820455,18818.51136,2947.517273
2021-08-01,9370.138571,19141.29667,2987.947143
2021-09-01,9324.709545,19376.87500,3036.018636


In [39]:
# Compute the `monthly_returns_df` DataFrame by chaining
# the pct_change function with the dropna function.
Monthly_Returns_df = Commodities_df.pct_change().dropna()
Monthly_Returns_df

,Copper,Nickel,Zinc
DATE,,,
1990-02-01,-0.002796,-0.011196,0.077280
1990-03-01,0.113084,0.328221,0.195122
1990-04-01,0.022670,-0.035394,0.012005
1990-05-01,0.020525,-0.026961,0.052788
1990-06-01,-0.057120,-0.031731,-0.033803
...,...,...,...
2021-07-01,-0.018759,0.046661,-0.001469
2021-08-01,-0.008537,0.017153,0.013717
2021-09-01,-0.004848,0.012307,0.016088


In [40]:
Monthly_Returns_df.describe()

,Copper,Nickel,Zinc
count,382.000000,382.000000,382.000000
mean,0.005530,0.005808,0.004336
std,0.060198,0.078993,0.061061
min,-0.298235,-0.317008,-0.253088
25%,-0.026170,-0.048926,-0.030183
50%,0.005192,-0.001535,0.003016
75%,0.036564,0.057991,0.040769
max,0.258375,0.328221,0.263915
